```bash
date
    Thu Apr 10 00:00:00 PM CEST 2025

cat /etc/os-release 
    PRETTY_NAME="Debian GNU/Linux 12 (bookworm)"
    NAME="Debian GNU/Linux"
    VERSION_ID="12"
    VERSION="12 (bookworm)"
    VERSION_CODENAME=bookworm
    ID=debian

python3 --version
    Python 3.11.2 
```

[logging — Logging facility for Python](https://docs.python.org/3.11/library/logging.html#module-logging)
[python logging in jupyter notebooks](https://www.mineo.app/blog-page/python-logging-in-jupyter-notebooks)